<a href="https://colab.research.google.com/github/chiyeon01/AI-Study/blob/main/Oxpet_Dataset2UltralyticsYOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ultralytics/yolov3.git

Cloning into 'yolov3'...
remote: Enumerating objects: 11749, done.
remote: Counting objects: 100% (1671/1671), done.
remote: Compressing objects: 100% (777/777), done.
remote: Total 11749 (delta 1197), reused 1260 (delta 893), pack-reused 10078 (from 1)
Receiving objects: 100% (11749/11749), 10.31 MiB | 15.62 MiB/s, done.
Resolving deltas: 100% (7996/7996), done.


In [2]:
!cd yolov3;pip install -qr requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.1/871.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.1 MB/s eta 0:00:00


In [3]:
!pwd

/content


### Dataset는 Oxford Pet Dataset 이용
* Dataset download : [https://www.robots.ox.ac.uk/~vgg/data/pets/](https://www.robots.ox.ac.uk/~vgg/data/pets/)

In [4]:
!mkdir /content/oxford_dataset

In [5]:
!wget https://thor.robots.ox.ac.uk/~vgg/data/pets/images.tar.gz
!wget https://thor.robots.ox.ac.uk/~vgg/data/pets/annotations.tar.gz

--2024-08-26 13:01:04--  https://thor.robots.ox.ac.uk/~vgg/data/pets/images.tar.gz
Resolving thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)... 129.67.95.98
Connecting to thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)|129.67.95.98|:443... connected.
HTTP request sent, awaiting response... 308 Permanent Redirect
Location: https://thor.robots.ox.ac.uk/pets/images.tar.gz [following]
--2024-08-26 13:01:05--  https://thor.robots.ox.ac.uk/pets/images.tar.gz
Reusing existing connection to thor.robots.ox.ac.uk:443.
HTTP request sent, awaiting response... 200 OK
Length: 791918971 (755M) [application/octet-stream]
Saving to: ‘images.tar.gz’

images.tar.gz       100%[===================>] 755.23M  21.7MB/s    in 36s     

2024-08-26 13:01:41 (20.9 MB/s) - ‘images.tar.gz’ saved [791918971/791918971]

--2024-08-26 13:01:42--  https://thor.robots.ox.ac.uk/~vgg/data/pets/annotations.tar.gz
Resolving thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)... 129.67.95.98
Connecting to thor.robots.ox.ac.uk (thor.robots.o

In [ ]:
!mkdir /content/data
!tar -xvf images.tar.gz -C /content/data
!tar -xvf annotations.tar.gz -C /content/data

In [7]:
# oxford dataset을 ultralytics yolo 형식으로 만들기(images <-> labels)

!cd oxford_dataset; mkdir ./images; mkdir ./labels
!cd oxford_dataset/images; mkdir ./train; mkdir ./val
!cd oxford_dataset/labels; mkdir ./train; mkdir ./val

In [8]:
!cd ~
!pwd

/content


In [9]:
# df형식으로 바꾸는 이유는 한번에 itteration하기 위해

import pandas as pd

oxford_dataset = pd.read_csv('/content/data/annotations/trainval.txt', sep=' ', names=['file_name', 'class_id', 'etc1', 'etc2'], header=None)
oxford_dataset.head()

,file_name,class_id,etc1,etc2
0,Abyssinian_100,1,1,1
1,Abyssinian_101,1,1,1
2,Abyssinian_102,1,1,1
3,Abyssinian_103,1,1,1
4,Abyssinian_104,1,1,1


In [10]:
# dataframe에 annotation file path / image file path 정보를 넣어줌.
# 이중에 annotation file path가 실제로 존재하지 않지만 txt에 넣어져 있는 경우도 있으므로 remove_no_ann_df를 이용해 제거해줌.
# 그렇게 만든 dataframe을 train/test용으로 나누어줌.

import os
from sklearn.model_selection import train_test_split

img_prefix = '/content/data/images/'
ann_prefix = '/content/data/annotations/xmls/'

def make_train_val_dataframe(df, img_prefix, ann_prefix, test_size=0.1):

  df['class_name'] = df['file_name'].apply(lambda x:x[:x.rfind('_')])
  df['image_file_path'] = img_prefix + df['file_name'] + '.jpg'
  df['annotation_file_path'] = ann_prefix + df['file_name'] + '.xml'

  df = remove_no_ann_df(df)
  train_df, test_df = train_test_split(df,test_size=test_size, stratify=df['class_id'])
  return train_df, test_df

def remove_no_ann_df(df):
  remove_df_idx = []

  for index, d in df.iterrows():
    ann_file = d['annotation_file_path']
    img_file = d['image_file_path']
    if not os.path.exists(ann_file):
      print('##### index:', index, ann_file, '가 존재하지 않아서 Dataframe에서 삭제함')
      remove_df_idx.append(index)
    elif not os.path.exists(img_file):
      print('##### index:', index, img_file, '가 존재하지 않아서 Dataframe에서 삭제함')
      remove_df_idx.append(index)
  df = df.drop(remove_df_idx, axis=0, inplace=False)
  return df

In [11]:
train_df, test_df = make_train_val_dataframe(oxford_dataset, img_prefix, ann_prefix)

##### index: 4 /content/data/annotations/xmls/Abyssinian_104.xml 가 존재하지 않아서 Dataframe에서 삭제함
##### index: 262 /content/data/annotations/xmls/Bengal_111.xml 가 존재하지 않아서 Dataframe에서 삭제함
##### index: 1456 /content/data/annotations/xmls/samoyed_10.xml 가 존재하지 않아서 Dataframe에서 삭제함
##### index: 2128 /content/data/annotations/xmls/Bengal_175.xml 가 존재하지 않아서 Dataframe에서 삭제함
##### index: 2395 /content/data/annotations/xmls/Egyptian_Mau_14.xml 가 존재하지 않아서 Dataframe에서 삭제함
##### index: 2402 /content/data/annotations/xmls/Egyptian_Mau_156.xml 가 존재하지 않아서 Dataframe에서 삭제함
##### index: 2427 /content/data/annotations/xmls/Egyptian_Mau_186.xml 가 존재하지 않아서 Dataframe에서 삭제함
##### index: 3177 /content/data/annotations/xmls/Ragdoll_199.xml 가 존재하지 않아서 Dataframe에서 삭제함
##### index: 3246 /content/data/annotations/xmls/saint_bernard_15.xml 가 존재하지 않아서 Dataframe에서 삭제함


In [12]:
train_df.shape, test_df.shape

((3303, 7), (368, 7))

In [13]:
train_df.head()

,file_name,class_id,etc1,etc2,class_name,image_file_path,annotation_file_path
3036,pomeranian_150,25,2,17,pomeranian,/content/data/images/pomeranian_150.jpg,/content/data/annotations/xmls/pomeranian_150.xml
2483,english_cocker_spaniel_18,13,2,7,english_cocker_spaniel,/content/data/images/english_cocker_spaniel_18...,/content/data/annotations/xmls/english_cocker_...
1965,american_pit_bull_terrier_163,3,2,2,american_pit_bull_terrier,/content/data/images/american_pit_bull_terrier...,/content/data/annotations/xmls/american_pit_bu...
1049,miniature_pinscher_103,22,2,15,miniature_pinscher,/content/data/images/miniature_pinscher_103.jpg,/content/data/annotations/xmls/miniature_pinsc...
1141,newfoundland_141,23,2,16,newfoundland,/content/data/images/newfoundland_141.jpg,/content/data/annotations/xmls/newfoundland_14...


In [58]:
train_df['class_id'].value_counts()

,count
class_id,
25,90
3,90
22,90
20,90
5,90
26,90
28,90
34,90
4,90


In [59]:
import xml.etree.ElementTree as ET

def xml2txt(df, type):
  for index, d in df.iterrows():
    output_path_txt = '/content/oxford_dataset/labels/' + type + '/' + d['file_name'] + '.txt'
    with open(output_path_txt, 'w') as write_path:
        ann_file_path = d['annotation_file_path']
        class_id = d['class_id'] - 1 # class_id - 1을 해줘야함
        tree = ET.parse(ann_file_path)
        root = tree.getroot()
        size_root = root.find('size')
        cols = int(size_root.find('width').text)
        rows = int(size_root.find('height').text)

        for ob in root.findall('object'):
          box = ob.find('bndbox')
          x1 = int(box.find('xmin').text)
          y1 = int(box.find('ymin').text)
          x2 = int(box.find('xmax').text)
          y2 = int(box.find('ymax').text)

          center_x = ((x1 + x2)/2.0)/cols
          center_y = ((y1 + y2)/2.0)/rows
          width = (x2 - x1)/cols
          height = (y2 - y1)/rows
          input_text = "{} {:.4f} {:.4f} {:.4f} {:.4f}".format(class_id, center_x, center_y, width, height)
          write_path.write(input_text + '\n')

def img2img(df, output_path):
  for index, d in df.iterrows():
    image_file_path = d['image_file_path']
    #print(os.path.exists(image_file_path))
    !cp "$image_file_path" "$output_path" # cp에 변수를 넣을땐 '$'를 넣어줘야함.

In [60]:
output_path_txt_train = '/content/oxford_dataset/labels/train'
output_path_txt_val = '/content/oxford_dataset/labels/val'
output_path_train = '/content/oxford_dataset/images/train'
output_path_val = '/content/oxford_dataset/images/val'ㅋ

In [61]:
xml2txt(train_df, 'train')
xml2txt(test_df, 'val')
img2img(train_df, output_path_train)
img2img(test_df, output_path_val)

In [62]:
print(os.path.exists(output_path_train))

True


In [63]:
!cd /content;pwd

/content


yaml : [https://github.com/chulminkw/DLCV/blob/master/data/util/ox_pet.yaml](https://github.com/chulminkw/DLCV/blob/master/data/util/ox_pet.yaml)

In [64]:
!wget https://raw.githubusercontent.com/chulminkw/DLCV/master/data/util/ox_pet.yaml

--2024-08-26 13:55:34--  https://raw.githubusercontent.com/chulminkw/DLCV/master/data/util/ox_pet.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 754 [text/plain]
Saving to: ‘ox_pet.yaml.3’

ox_pet.yaml.3       100%[===================>]     754  --.-KB/s    in 0s      

2024-08-26 13:55:34 (34.9 MB/s) - ‘ox_pet.yaml.3’ saved [754/754]



In [65]:
!cd /content/yolov3; pwd

/content/yolov3


In [66]:
!pwd

/content


In [67]:
# model train
!cd /content/yolov3; python train.py --img 640 --batch 16 --epochs 30 --data /content/ox_pet.yaml --weights yolov3.pt --project=/content --name pet_model --exist-ok

2024-08-26 13:55:39.452295: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-26 13:55:39.472142: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-26 13:55:39.478149: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov3.pt, cfg=, data=/content/ox_pet.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, pro